In [ ]:
data = pd.read_csv('/content/drive/MyDrive/team cat/02.입력 데이터처리 출력 과정/가사처리및분석/처리데이터/data_kor_2000-2499.csv', index_col=0)

In [ ]:
data = data.iloc[:27000,]

In [ ]:
data = data[~data['Genre'].str.contains('CCM')]
data = data.dropna()

In [ ]:
# data.drop(data.loc[data['Genre']=='J-POP'].index, inplace=True)

In [ ]:
lyr_result = data['lyr_kor_notBlack']

In [ ]:
tagging_ex=[]
for i in tq(lyr_result):
  tagging_ex.append(mecab.pos(i))

100%|██████████| 27000/27000 [00:43<00:00, 623.77it/s]


In [ ]:
stopword=[]
for i in tq(tagging_ex):
  for word, tag in i:
    if tag in 'SL' or tag in 'SF' or tag in 'SP' or tag in 'SY' or tag in 'SE' or tag in 'SS' or tag in 'SE' or tag in 'SO' or tag in 'SW' or tag in 'UN' or tag in 'OH'or tag in 'ON'or tag in 'OL'or tag in 'IC':
      stopword.append(word)

100%|██████████| 27000/27000 [00:03<00:00, 6969.47it/s]


In [ ]:
stop_set = set(stopword)
stopword = list(stop_set)

In [ ]:
tagging_result=[]
for i in tq(lyr_result):
  tagging_result.append([word for word in mecab.morphs(i) if not word in stopword])

100%|██████████| 27000/27000 [50:40<00:00,  8.88it/s]


In [ ]:
num_list, w2n_dic, n2w_dic = word2num(tagging_result)

In [ ]:
import operator

In [ ]:
nul_index = []
for i in range(len(num_list)):
    if not num_list[i]:
        nul_index.append(i)
        
for i in nul_index:
    del num_list[i]
    num_list.insert(0,'null')
    
for i in range(len(num_list)):
    if num_list[0]=='null':
        num_list.pop(0)

for i in range(len(num_list)):
    if not num_list[i]:
        print(i)

In [ ]:
num_list = sequence.pad_sequences(num_list, maxlen=300)

In [ ]:
import numpy as np

In [ ]:
# emotion1 = []
# emotion2 = []
# emotion_per = []
# emotion_label = {'즐거움':0,'슬픔':1,'불안':2,'사랑':3,'분노':4,'외로움':5}
anger=[]
unrest = []
love = []
sad = []
lonely = []
happy = []

for i in tq(num_list):
  lyr_emotion = {'happy':0, 'sad':0, 'unrest':0, 'love':0, 'anger':0, 'lonely':0}
  for j in load_model.predict(i.reshape(1,300)):
    k = 0
    for emo in lyr_emotion:
      lyr_emotion[emo] += j[k]
      k += 1
  anger.append(lyr_emotion['anger'])
  unrest.append(lyr_emotion['unrest'])
  love.append(lyr_emotion['love'])
  sad.append(lyr_emotion['sad'])
  lonely.append(lyr_emotion['lonely'])
  happy.append(lyr_emotion['happy'])

  # emo_per = sorted(lyr_emotion.items(), key=operator.itemgetter(1))
  # emotion_per.append(emo_per)
  # emotion1.append(emo_per[3][0])
  # emotion2.append(emo_per[2][0])

100%|██████████| 27000/27000 [50:19<00:00,  8.94it/s]


In [ ]:
lyr_emotion['anger']

1.983312131415005e-06

In [ ]:
data_dic_emo = data

In [ ]:
# data_dic_emo['emotion1'] = emotion1
# data_dic_emo['emotion2'] = emotion2
# data_dic_emo['emotion_per'] = emotion_per
data_dic_emo['happy_score'] = happy
data_dic_emo['sad_score'] = sad
data_dic_emo['unrest_score'] = unrest
data_dic_emo['love_score'] = love
data_dic_emo['anger_score'] = anger
data_dic_emo['lonely_score'] = lonely

In [ ]:
data_dic_emo

In [ ]:
data_dic_emo.drop(['Album','Release_Date','Genre','lyr_kor','lyr_kor_notBlack'],axis=1, inplace=True)

In [ ]:
data_dic_emo[data_dic_emo['happy_sum']>0.5]

In [ ]:
data_dic_emo.to_csv('/content/drive/MyDrive/team cat/02.입력 데이터처리 출력 과정/가사처리및분석/처리데이터/data_emo_2000-2499_1.csv')

# 새 섹션

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()

In [ ]:

def sentiment_predict(new_sentence):
  new_token=[word for word in mecab.morphs(new_sentence) if not word in stopword]
  new_sequences = tokenizer.texts_to_sequences([new_token])
  new_pad = pad_sequences(new_sequences, maxlen=300)
  score = load_model.predict(new_pad[0].reshape(1,300))
  



In [ ]:
for i in tq(data['lyr_kor_notBlack']):
  data['predict'] = sentiment_predict(i)

100%|██████████| 14827/14827 [23:47<00:00, 10.39it/s]


In [ ]:
data['lyr_kor_notBlack']

0        ангел энергия Снегурочка Дед Мороз ангел энерг...
8                                  ？？짹 ？？짹 ？？짹 ？？짹 ？？짹 ？？짹
11       너와 내가 함께한 순간들그 모든 게 하나같이 다 선명해아무리 아무리 많은 말 꺼내고...
12       휴 요즘 나 고민이 참 많아요말할 곳도 없고 너무 답답해요축 늘어진 어깨 토닥여줬던...
13       원하면 어디든 날씨도 그래서 준비했어이 세상의 비상구너와 날 가로막는 그 창문허물고...
                               ...                        
30671    여자야 오늘밤 유혹하지마또다른 사랑에 상처주지마너의 그 앙큼한 사랑때문에밤마다 술잔...
30672    너 없는 세상 나 살수 없어언제부턴가 내 안에 너 있어난 너의 사랑 먹으며 살고 싶...
30673    그때여레게스카 러브피플이 한 자리에 모여한 마음 한 사랑으로놀아 보든 것이었다한 사...
30675    눈웃음 살랑살랑 내 맘 날려보고눈싸움 하듯 네게 레이절 쏘아봐도전해지지 않는 맘 오...
30676    다 같은 색 이야 그게 다 그거야아무 재미 없어 지겨워이유가 뭘까 있나 할까진지한 ...
Name: lyr_kor_notBlack, Length: 14827, dtype: object